# Working with Data:
### New York City 311 Requests Made in 2021

`311_Service_Requests_2021.csv` contains data on all Service Requests (SR) made to NYC's 311 in the first half of January, 2021. Select fields have been retained, as described below. Full data are available at https://data.cityofnewyork.us/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9


### This Notebook
Contains working code and examples of basic data analysis. Remember: There is more than one correct way to code! This solutions guide is intended to help if you get stuck, but your code doesn't have to be implemented a certain way.


### Data Fields:

`Unique Key`: Unique identifier of a Service Request (SR) in the open data set

`Created Date`: Date SR was created

`Closed Date`: Date SR was closed by responding agency

`Agency`: Acronym of responding City Government Agency

`Agency Name`: Full Agency name of responding City Government Agency

`Complaint Type`: This is the first level of a hierarchy identifying the topic of the incident or condition. Complaint Type may have a corresponding Descriptor (below) or may stand alone.

`Descriptor`: This is associated to the Complaint Type, and provides further detail on the incident or condition. Descriptor values are dependent on the Complaint Type, and are not always required in SR.

`Location Type`: Describes the type of location used in the address information

`Incident Zip`: Incident location zip code, provided by geo validation.

`Address Type`: Type of incident location information available.

`City`: City of the incident location provided by geovalidation.

`Landmark`: If the incident location is identified as a Landmark the name of the landmark will display here

`Facility Type`: If available, this field describes the type of city facility associated to the SR

`Status`: Status of SR submitted

`Due Date`: Date when responding agency is expected to update the SR. This is based on the Complaint Type and internal Service Level Agreements (SLAs).

`Resolution Description`: Describes the last action taken on the SR by the responding agency. May describe next or future steps.

`Resolution Action Updated Date`: Date when responding agency last updated the SR.

`BBL`: Borough Block and Lot, provided by geovalidation. Parcel number to identify the location of location of buildings and properties in NYC.

`Borough`: Provided by the submitter and confirmed by geovalidation.

`Open Data Channel Type`: Indicates how the SR was submitted to 311. i.e. By Phone, Online, Mobile, Other or Unknown.


# Intro and Setup

(This code runs as expected)

In [1]:
# --------- Global variables ------------#
filename = '311_Service_Requests_2021.txt'

In [2]:
# Load data as a dictionary of form:
# {key: [values]}

# NOTE: Pandas (https://pandas.pydata.org) is a common package for working with 
#       table data. Here, though, we'll do all analysis in base Python

data = dict()

with open(filename, 'r') as fp:
    
    # for each line of data
    for i, line in enumerate(fp.readlines()):
        
        # strip any white space and split using tabs ('\t')
        line = line.strip().split('\t')
        
        if i == 0:
            # the first row contains header values
            header = line
            
            # make these header values the keys of the dictionary
            for key in header:
                data.setdefault(key, list())
                
        else:
            # for all lines past the first, append value to appropriate header list
            for key, value in zip(header, line):
                data[key].append(value)
                
# Now we have a dictionary where the i index of every value list represents the same row of data

In [3]:
# Look at sample of data

for key, values in data.items():
    print(key, values[:2]) # print first two "rows" of data

Unique Key ['49499840', '49502362']
Created Date ['01/09/2021 11:56:15 AM', '01/09/2021 10:49:15 AM']
Closed Date ['01/14/2021 06:41:59 AM', '']
Agency ['DOT', 'DOT']
Agency Name ['Department of Transportation', 'Department of Transportation']
Complaint Type ['Broken Parking Meter', 'Highway Sign - Missing']
Descriptor ['Out of Order', 'Other/Unknown']
Location Type ['Street', 'Highway']
Incident Zip ['', '']
Address Type ['', '']
City ['', '']
Landmark ['', '']
Facility Type ['', '']
Status ['Closed', 'In Progress']
Due Date ['', '']
Resolution Description ['"The Department of Transportation inspected the condition you reported. You can find additional information in the ""Notes to Customer"" field."', '']
Resolution Action Updated Date ['01/14/2021 11:42:02 AM', '']
BBL ['', '']
Borough ['Unspecified', 'Unspecified']
Open Data Channel Type ['ONLINE', 'ONLINE']


# Questions and Exercises

This code will ask and answer questions about the dataset. This Notebook contains **working solutions**.

In [4]:
# QUESTION: How many Service Requests are in this dataset?
    
print('There are {} service requests.'.format(len(data['Unique Key'])))
# Should be 90271 requests

There are 90271 service requests.


In [5]:
# QUESTION: What is the most recent day of created requests included in this dataset?

# TIPS: the dates are *not* in order
#       All Service Requests were created in Jan 2021 (eg, month and year don't change)
#       'Created Date' is a string of format: '01/09/2021 11:56:15 AM'

last_day = 0 

for timestamp in data['Created Date']:   
    date, time, am_pm = timestamp.split() # splits on space by default
            
    month, day, year = date.split('/')
            
    if int(day) > last_day:
        last_day = int(day) # keep record of highest day
            
print('The last date is 1/{}/21'.format(last_day))
# Should be 1/16/21

The last date is 1/16/21


In [6]:
# QUESTION: What are the top five most common "Complaint Type"s?

complaint_counts = dict()

for complaint in data['Complaint Type']:
    # for each new type we see, add to dict with default of 0 occurances
    complaint_counts.setdefault(complaint, 0)
    
    # iterate count
    complaint_counts[complaint] += 1
    
# sort this dictionary by value from highest count to lowest
top_complaints = dict((k, v) for k, v in sorted(complaint_counts.items(),
                  key=lambda item: item[1], reverse=True))
# read more about lambda functions: https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions

print('Top complaint types:')
for i, (key, val) in enumerate(top_complaints.items()):
    if i < 5:
        print('   {}. {}: {} complaints'.format(i + 1, key, val))

Top complaint types:
   1. HEAT/HOT WATER: 13270 complaints
   2. Noise - Residential: 11775 complaints
   3. Illegal Parking: 8727 complaints
   4. Blocked Driveway: 4676 complaints
   5. UNSANITARY CONDITION: 3013 complaints


In [7]:
# QUESTION: What percent of Service Requests are closed?
closed = data['Status'].count('Closed')

print('{} ({:.2%}) of service requests are closed'.format(closed, closed/len(data['Status'])))

69390 (76.87%) of service requests are closed


In [8]:
# QUESTION: How often are different "Open Data Channel Type" used to submit service requests?

for channel in set(data['Open Data Channel Type']):
    count = data['Open Data Channel Type'].count(channel)

    print('Channel {} is used {} ({:.2%}) of the time'.format(channel, 
                                                              count, 
                                                              count/len(data['Open Data Channel Type'])))

Channel MOBILE is used 14723 (16.31%) of the time
Channel UNKNOWN is used 9652 (10.69%) of the time
Channel ONLINE is used 34310 (38.01%) of the time
Channel PHONE is used 31585 (34.99%) of the time
Channel OTHER is used 1 (0.00%) of the time


In [9]:
# QUESTION: What is the average time (in days) a request remains open for?
#           What is the maximum time? Minumum time?

days_open = list()

for i, timestamp in enumerate(data['Closed Date']):
    
    # if closed date is not null
    if timestamp != '':
        date, time, am_pm = timestamp.split()
        
        month, close_day, year = date.split('/')
        
        # get associated open date
        open_date = data['Created Date'][i]
        
        month, open_day, year = open_date.split('/')
        
        # record days between closing and opening of request
        days_open.append(int(close_day) - int(open_day))
        
print('Closed requests were open for an average of {:.2} days'.format(sum(days_open)/len(days_open)))
print('The shortest time open was {} days'.format(min(days_open)))
print('The longest time open was {} days'.format(max(days_open)))

# NOTE: Shortest time returns...-8 days. See next question

Closed requests were open for an average of 1.1 days
The shortest time open was -8 days
The longest time open was 27 days


In [10]:
# QUESTION: How have requests been closed for a negative number of days??

# to start, record index where closed date is before open date
weird_indexes = dict()

for i, timestamp in enumerate(data['Closed Date']):
    if timestamp != '':
        date, time, __ = timestamp.split()
        
        month, close_day, year = date.split('/')
        
        # open date
        open_date = data['Created Date'][i]
        
        month, open_day, year = open_date.split('/')
        
        diff = int(close_day) - int(open_day)
        
        if diff < 0:
            weird_indexes[i] = diff
            
print('{} entries closed before they were opened'.format(len(weird_indexes)))

417 entries closed before they were opened


In [11]:
# see just the first 5 records
for index in list(weird_indexes.keys())[:5]:
    print('Created:', data['Created Date'][index])
    print('Closed:', data['Closed Date'][index])
    print('Status:', data['Status'][index])
    print()

Created: 01/07/2021 11:29:00 AM
Closed: 01/06/2021 12:45:00 AM
Status: Pending

Created: 01/07/2021 11:30:00 AM
Closed: 01/06/2021 08:00:00 AM
Status: Pending

Created: 01/04/2021 09:23:00 AM
Closed: 01/03/2021 12:40:00 AM
Status: Pending

Created: 01/04/2021 09:34:00 AM
Closed: 01/03/2021 12:30:00 AM
Status: Pending

Created: 01/04/2021 08:50:00 AM
Closed: 01/03/2021 12:20:00 AM
Status: Pending



In [12]:
# look at sample of records with 8 day difference
display = 5

for index, diff in weird_indexes.items():
    if diff == -8 and display >0:
        print('Created:', data['Created Date'][index])
        print('Closed:', data['Closed Date'][index])
        print('Status:', data['Status'][index])
        print()
        display -= 1

Created: 01/09/2021 08:06:00 AM
Closed: 01/01/1900 12:00:00 AM
Status: Open

Created: 01/09/2021 08:54:00 AM
Closed: 01/01/1900 12:00:00 AM
Status: Open

Created: 01/09/2021 09:14:00 AM
Closed: 01/01/1900 12:00:00 AM
Status: Open

Created: 01/09/2021 08:51:00 AM
Closed: 01/01/1900 12:00:00 AM
Status: Open

Created: 01/09/2021 09:11:00 AM
Closed: 01/01/1900 12:00:00 AM
Status: Open



In [14]:
# TAKE 2:   What is the average time (in days) a request remains open for?
#           What is the maximum time? Minumum time?

days_open = list()

for i, status in enumerate(data['Status']):
    if status == 'Closed':
        close_timestamp = data['Closed Date'][i]
        
        if close_timestamp != '':
            close_date, time, __ = close_timestamp.split()

            month, close_day, year = close_date.split('/')

            # open date
            open_date = data['Created Date'][i]

            month, open_day, year = open_date.split('/')

            days_open.append(int(close_day) - int(open_day))
        
print('Closed requests were open for an average of {:.2} days'.format(sum(days_open)/len(days_open)))
print('The shortest time open was {} days'.format(min(days_open)))
print('The longest time open was {} days'.format(max(days_open)))

Closed requests were open for an average of 1.1 days
The shortest time open was 0 days
The longest time open was 14 days
